In [16]:

from comet_ml import Experiment
import GPUtil
import torch

import os
import codecarbon
from iesta.machine_learning.dataloader import IESTAData, LABELS
from iesta.machine_learning.huggingface_loader import IESTAHuggingFace
from nlpaf.transformers.text_classification import TextClassification
from dotenv import load_dotenv, find_dotenv
import argparse
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict


In [6]:
data_object = IESTAData(ideology="liberal", keep_labels = LABELS.EFF_INEFF)
huggingface_dataset = IESTAHuggingFace(data_object)

dataset_name = huggingface_dataset.get_dataset_name(is_for_style_classifier=True)
dataset_name

'notaphoenix/debateorg_w_effect_for_liberal_subset'

Found cached dataset parquet (/home/elba_ro/.cache/huggingface/datasets/notaphoenix___parquet/notaphoenix--debateorg_w_effect_for_liberal_subset-2510e01b5ef739a4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [26]:
"adsaA".lower()

'adsaa'

In [62]:
import re
a_string = '!hi. wh?at is the weat[h]er lik?e.'
new_string = re.sub(r'[^\w\s]', '', a_string)

print(new_string)

with open("../data/dismiss_text.txt", "r") as dismissedf:
    dissmiss_arr = list(pd.Series(dismissedf.read().splitlines()).str.lower())
    dissmiss_arr = list(set([re.sub(r'[^\w\s]', '', x) for x in dissmiss_arr]))

dissmiss_arr

hi what is the weather like


['',
 'arguments extended',
 'url',
 'round has not been posted',
 'accept',
 'i win',
 'ok',
 'i extend my argument',
 'yes',
 'extend',
 'vote con',
 'extend all arguments',
 'extended',
 'i accept',
 'vote pro',
 'first round is acceptance',
 'i extend',
 'i extend my arguments',
 'content removed by moderator',
 'please extend my arguments',
 'all points extended',
 'accepted',
 'challenge accepted',
 'ff',
 'i accept this debate',
 'extend my arguments',
 'extend vote con',
 'extend arguments',
 'i extend all arguments']

In [64]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
def _apply_no_punc(row):
    row["text_no_punc"] = re.sub(r'[^\w\s]', '', row["text"])
    return row

def profile_datasets(ds_name:str = "debateorg_w_effect_for_liberal", lower:bool = False):
    dataset_dict: DatasetDict = load_dataset(f"notaphoenix/{ds_name}")
    dfs = []
    for split, ds in dataset_dict.items():
        print(split)
        dfs.append(ds.to_pandas())

    df = pd.concat(dfs)
    len(df)
        #profile = ProfileReport(df, title="Profiling Report")
    #df["text_low"] = df["text"].str.lower()
    dissmiss_arr = []
    with open("../data/dismiss_text.txt", "r") as dismissedf:
        dissmiss_arr = list(pd.Series(dismissedf.read().splitlines()).str.lower())
        dissmiss_arr = list(set([re.sub(r'[^\w\s]', '', x) for x in dissmiss_arr]))
    
    
    df = df.apply(_apply_no_punc, axis=1)
    df = df[~df["text_no_punc"].str.lower().isin(dissmiss_arr)]
    if lower:
        df["text_low"] = df["text"].str.lower()
        profile = ProfileReport(df[["text_low"]], title="Profiling Report")
        profile.to_file(f"../data/profilers/{ds_name}_low.html")
    else:
        profile = ProfileReport(df[["text"]], title="Profiling Report")
        profile.to_file(f"../data/profilers/{ds_name}.html")
    return profile

In [65]:
dataset_names = ["debateorg_w_effect_for_liberal_subset",
                 "debateorg_w_effect_for_conservative_subset",
                 "debateorg_w_effect_for_liberal",
                 "debateorg_w_effect_for_conservative"]



for ds_name in dataset_names:
    profile_datasets(ds_name, lower=True)
    

Found cached dataset parquet (/home/elba_ro/.cache/huggingface/datasets/notaphoenix___parquet/notaphoenix--debateorg_w_effect_for_liberal_subset-2510e01b5ef739a4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

validation
test
training


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

...

/home/elba_ro/repos/github/conf22-style-transfer/iesta_venv/lib/python3.8/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

['i accept.',
 'round has not been posted',
 'vote con.',
 'first round is acceptance.',
 'i accept',
 'vote pro',
 'extend',
 'extend.',
 'vote con',
 'vote pro.',
 'extend all arguments.',
 '.',
 'i accept this debate.',
 'extend my arguments.',
 '<url>...',
 'extend all arguments']